In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
sales_df = pd.read_excel('data/train_sales.xlsx')
promo_df = pd.read_excel('data/train_promo.xlsx')
geography_df = pd.read_excel('data/Атрибуты Customer.xlsx')
weights_df = pd.read_excel('data/Веса DFU.xlsx')

In [ ]:
sales_df['Customer'].unique()

In [ ]:
sales_df['DFU'].unique()

In [ ]:
promo_df['DFU'].unique()

In [ ]:
sales_df[sales_df['DFU'] == 'Рис басмати 500 гр']['Customer'].unique()

In [ ]:
promo_df[promo_df['DFU'] == 'Рис басмати 500 гр']['Customer'].unique()

In [ ]:
for dfu_sales in sales_df['DFU'].unique():
    print(f"Кастомеры по продукту {dfu_sales}")
    print(f"train_sales: {sales_df[sales_df['DFU'] == dfu_sales]['Customer'].unique()}")
    print(f"train_promo: {promo_df[promo_df['DFU'] == dfu_sales]['Customer'].unique()}")
    print('\n')

In [ ]:
sales_df.head(10)

In [ ]:
promo_df.head(10)

Начнем с обработки выбросов (выбросом считаем, если значение больше 2 сигм - из встречи с зак)

In [ ]:
bpv_mean = sales_df["BPV"].mean()
bpv_std = sales_df["BPV"].std()
lower_bound = bpv_mean - 2 * bpv_std
upper_bound = bpv_mean + 2 * bpv_std

bpv_outliers = sales_df[(sales_df["BPV"] < lower_bound) | (sales_df["BPV"] > upper_bound)]

print(f"Outliers in BPV: {len(bpv_outliers)}")
bpv_outliers

Всего 31 выброс, это 1.5% от всех данных, можно удалить

In [ ]:
sales_df_cleaned = sales_df[(sales_df["BPV"] >= lower_bound) & (sales_df["BPV"] <= upper_bound)]
sales_df_cleaned = sales_df_cleaned[sales_df_cleaned["BPV"]>= 0]
print(f"Rows after cleaning: {len(sales_df_cleaned)}")

Посмотрим на выбросы в promo days on shelf

In [ ]:
promo_days_mean = promo_df["Promo Days on shelf"].mean()
promo_days_std = promo_df["Promo Days on shelf"].std()
lower_promo_bound = promo_days_mean - 2 * promo_days_std
upper_promo_bound = promo_days_mean + 2 * promo_days_std

promo_days_outliers = promo_df[(promo_df["Promo Days on shelf"] < lower_promo_bound) |
                               (promo_df["Promo Days on shelf"] > upper_promo_bound)]

print(f"Outliers Promo Days on shelf: {len(promo_days_outliers)}")
promo_days_outliers

2% от общих, тоже дропнем

In [ ]:
promo_df_cleaned = promo_df[(promo_df["Promo Days on shelf"] >= lower_promo_bound) &
                            (promo_df["Promo Days on shelf"] <= upper_promo_bound)]
promo_df_cleaned.drop(columns=["Promo №"], inplace=True)
print(f"Rows after cleaning: {len(promo_df_cleaned)}")

Теперь обработаем пропущенные значения в promo_df

In [ ]:
missing_promo_days = promo_df_cleaned[promo_df_cleaned["Promo Days on shelf"].isna()]

missing_promo_days

а таких у нас нет

Оставим даты не раньше 2017 года

In [ ]:
promo_df_cleaned = promo_df_cleaned[promo_df_cleaned["Start Date on shelf"] >= np.datetime64("2017-01-01")]

Переведем Units SoD в тонны

In [ ]:
promo_df_cleaned = pd.merge(promo_df_cleaned, weights_df, on="DFU", how="left")
promo_df_cleaned["SoD"] = promo_df_cleaned["Units SoD"] * promo_df_cleaned["Unit weight, tn"]
promo_df_cleaned.drop(columns=["Unit weight, tn"], inplace=True)
promo_df_cleaned

Добавим сезонность в sales

In [ ]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"

sales_df_cleaned["Season"] = sales_df_cleaned["Period"].apply(get_season)

Объединим с географией и типом Клиента

In [ ]:
sales_df_cleaned = pd.merge(sales_df_cleaned, geography_df, left_on="Customer", right_on="Клиент", how="left")
sales_df_cleaned.rename(columns={"Тип": "Type", "География": "Geography"}, inplace=True)

In [ ]:
sales_df_cleaned.drop(columns=["Клиент"], inplace=True)
sales_df_cleaned

In [ ]:
sales_df_cleaned['End of Period'] = sales_df_cleaned['Period'] + pd.Timedelta(days=6)

In [ ]:
sales_df_cleaned.info()

In [ ]:
promo_df_cleaned.info()

In [ ]:
promo_cust_1 = promo_df_cleaned[promo_df_cleaned['Customer'] == 1]

In [ ]:
promo_cust_1.info()

In [ ]:
#sales_df_cleaned.to_excel('sales_df_cleaned.xlsx')

In [ ]:
#promo_cust_1.to_excel('promo_cust_1.xlsx')

In [ ]:
sales_df_cleaned[sales_df_cleaned['BPV'] != sales_df_cleaned['Total Sell-in']]

In [ ]:
sales_df_cleaned[sales_df_cleaned['BPV'] == 0]

In [ ]:
sales_df_cleaned[(sales_df_cleaned['BPV'] == 0) & (sales_df_cleaned['Customer'] != 1)]

In [ ]:
sales_df_cleaned[(sales_df_cleaned['BPV'] == 0) & (sales_df_cleaned['Total Sell-in'] != 0)]

In [ ]:
sales_df_cleaned[(sales_df_cleaned['BPV'] != sales_df_cleaned['Total Sell-in']) & (sales_df_cleaned['Customer'] != 1)]

In [ ]:
promo_c1_weekdays = promo_cust_1.copy()

In [ ]:
promo_c1_weekdays.info()

In [ ]:
promo_c1_weekdays.to_excel('promo_c1_weekdays.xlsx')

Есть два датасета pandas:

sales_df_cleaned:
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DFU            2062 non-null   object        
 1   Customer       2062 non-null   int64         
 2   Period         2062 non-null   datetime64[ns]
 3   BPV            2062 non-null   float64       
 4   Total Sell-in  2062 non-null   float64       
 5   Season         2062 non-null   object        
 6   Type           2062 non-null   object        
 7   Geography      2062 non-null   object        
 8   End of Period  2062 non-null   datetime64[ns]

promo_cust_1:
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Promo                         183 non-null    int64         
 1   Customer                      183 non-null    int64         
 2   DFU                           183 non-null    object        
 3   Promo mechanic                183 non-null    int64         
 4   Start Date on shelf           183 non-null    datetime64[ns]
 5   Promo Days on shelf           183 non-null    float64       
 6   End Date on shelf             183 non-null    datetime64[ns]
 7   Shipment days to promo start  183 non-null    int64         
 8   First Date of shipment        183 non-null    datetime64[ns]
 9   End Date of shipment          183 non-null    datetime64[ns]
 10  Discount, %                   183 non-null    float64       
 11  Units SoD                     183 non-null    float64       
 12  SoD                           180 non-null    float64  


 В promo_cust_1 только один Customer = 1, он же есть в первом датасете. Нужно их соединить с помощью merge, но есть несколько проблем:

 Period и End of Period в sales - это понедельник и воскресенье соответственно, однако в promo_cust_1 "Start Date on shelf" может начинаться не с понедельника и заканчиваться (End Date on shelf) не в воскресенье:

 		Promo
weekday	end weekday

Monday -	Sunday	2


Sunday -	Saturday	2

Thursday:

-Friday 1

-Monday	2

-Saturday	1

-Thursday	20

-Wednesday	11

Tuesday:
-Monday	140

-Thursday	3

-Tuesday	1


Нужно соединить по следующей логике:

Если даты периода в строке sales пересекаются с promo не польностью, то нужно разделить на две строки - одна покрывает период без промо-акции, а другая - с ней. Также нужно учесть следующий пункт:

bpv == total sell-in и был промо - в так случае мы считаем, что это ложь и промо не было

# Merge

In [ ]:
sales_df_cleaned.info()

In [ ]:
sales_df_cleaned['Year-Week'] = sales_df_cleaned['Period'].dt.strftime('%Y-%W')
sales_df_cleaned.sort_values('Period')

In [ ]:
promo_cust_1.columns

In [ ]:
def expand_dates(row):
    return pd.DataFrame({
        'Promo': row['Promo'],
        'Customer': row['Customer'],
        'DFU': row['DFU'],
        'Promo mechanic': row['Promo mechanic'],
        'Start Date on shelf': row['Start Date on shelf'],
        'Promo Days on shelf': row['Promo Days on shelf'],
        'End Date on shelf': row['End Date on shelf'],
        'Shipment days to promo start': row['Shipment days to promo start'],
        'Date': pd.date_range(start=row['First Date of shipment'],
                              end=row['End Date of shipment']),
        'First Date of shipment': row['First Date of shipment'],
        'End Date of shipment': row['End Date of shipment'],
        'Discount, %': row['Discount, %'],
        'Units SoD': row['Units SoD'],
        'SoD': row['SoD'],
    })

# Генерация нового DataFrame с датами
expanded_df = pd.concat(promo_cust_1.apply(expand_dates, axis=1).to_list(), ignore_index=True)

In [ ]:
expanded_df.head(25)

In [ ]:
expanded_df['Year-Week'] = expanded_df['Date'].dt.strftime('%Y-%W')

expanded_df.head(25)

In [ ]:
expanded_unique = expanded_df.drop_duplicates(subset = ['Promo', 'DFU', 'Year-Week'], keep = 'first').reset_index(drop = True)

In [ ]:
sales_df_cleaned.info()

In [ ]:
expanded_unique.info()

In [ ]:
merged = pd.merge(
    left=sales_df_cleaned,
    right=expanded_unique,
    how='left',
    left_on=['Customer', 'DFU', 'Year-Week'],
    right_on=['Customer', 'DFU', 'Year-Week'],
)

In [ ]:
merged.to_excel('merged_sales_promo.xlsx')

In [ ]:
mask = (merged['BPV'] == merged['Total Sell-in']) & (merged['BPV'] > 0)
cols_to_clean = ['Promo', 'Promo mechanic',
       'Start Date on shelf', 'Promo Days on shelf', 'End Date on shelf',
       'Shipment days to promo start', 'Date', 'First Date of shipment',
       'End Date of shipment', 'Discount, %', 'Units SoD', 'SoD']

merged.loc[mask, cols_to_clean] = np.nan
merged.to_excel('merged_cleaned.xlsx')

# Моделинг, эксперименты

посмотрим на кейсы, когда у нас бпв не равен тотал селл ин и промо не было

In [ ]:
merged['Customer'].unique()

In [ ]:
merged_bpv_si_conflicts = merged[(merged['BPV'] != merged['Total Sell-in']) & (merged['Promo'].isna() == True)].copy()

In [ ]:
merged_bpv_si_conflicts['Customer'].unique()

посторим по конфликтным график - здесь в данных bpv != total sell in, но при этом промо НЕ было

In [ ]:
plot_data = merged_bpv_si_conflicts.sort_values(by="Period")
plot_data

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(plot_data["Period"], plot_data["BPV"], marker='o', label="BPV", linestyle='-')
plt.plot(plot_data["Period"], plot_data["Total Sell-in"], marker='s', label="Total Sell-in", linestyle='--')

plt.xlabel("Period")
plt.ylabel("Value")
plt.title("Изменение BPV и Total Sell-in по времени")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()

построим еще для каждого dfu

In [ ]:
import math

dfu_values = plot_data["DFU"].unique()
num_dfu = len(dfu_values)

num_cols = 3
num_rows = math.ceil(num_dfu / num_cols)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
axes = axes.flatten()

for i, dfu in enumerate(dfu_values):
    ax = axes[i]
    dfu_data = plot_data[plot_data["DFU"] == dfu]
    
    ax.plot(dfu_data["Period"], dfu_data["BPV"], marker='o', linestyle='-', label="BPV")
    ax.plot(dfu_data["Period"], dfu_data["Total Sell-in"], marker='s', linestyle='--', label="Total Sell-in")
    
    ax.set_title(f"DFU: {dfu}")
    ax.set_xlabel("Period")
    ax.set_ylabel("Value")
    ax.legend()
    ax.grid(True)
    ax.tick_params(axis='x', rotation=45)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
merged[merged['Promo'].isna() == False]

In [ ]:
merged[merged['Promo'].isna() == False].to_excel('merged_si_conflicts.xlsx')

# EDA к моделингу

попробуем сделать следующее: в конфликтных случаях - если у нас bpv != total sell in и промо не было, то просто делаем bpv равным total sell in

In [ ]:
df = merged.copy()
df.head(5)

ВОПРОС: в BPV пишем total sell in, поскольку нет данных по промо?

промо сдвинулось. нужно поправку делать. если bpv == 0 и total sell in не равен, то точно было промо. вопрос что делать с граничными. в воскр не бывает отгрузок - нудно смотреть в граничных неделях

In [ ]:
df.loc[(df['BPV'] != df['Total Sell-in']) & (df['Promo'].isna()), 'BPV'] = df['Total Sell-in']
df.head(5)

Почистим данные и подумаем что делать с нанами

In [ ]:
df.info()

определять промо по bpv и sell in

In [ ]:
df[(df['SoD'].isna()) & (df['Promo'].isna() == False)]

In [ ]:
df = df.drop(df[(df['SoD'].isna()) & (df['Promo'].notna())].index)
df.info()

In [ ]:
promo_columns = ["Promo", "Promo mechanic", "Promo Days on shelf", 
                 "Shipment days to promo start", "Discount, %", 
                 "Units SoD", "SoD"]
df[promo_columns] = df[promo_columns].fillna(0)

date_columns = ["Start Date on shelf", "End Date on shelf", "Date",
                "First Date of shipment", "End Date of shipment"]
df[date_columns] = df[date_columns].fillna(pd.to_datetime("1900-01-01"))

df

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates(subset=['DFU', 'Customer', 'Period', 'BPV', 'Total Sell-in'])
df.info()

In [ ]:
df.to_excel('dataframe.xlsx')

Теперь надо разобраться с SoD

Ожидаемая картина:
1) Во всем датасете SoD + BPV = Total Sell In

Условие соблюдается только в 1500 строках, а не во всем датасете

In [ ]:
df[df['SoD'] + df['BPV'] == df['Total Sell-in']]

Посмотрим на те случаи, когда условие не соблюдается

Вот тут один ктсти прям ништяк есть

Если не равно , то просто 

In [ ]:
df[(df['Promo' ] != 0) & (df['BPV'] + df['SoD'] == df['Total Sell-in'])]

In [ ]:
df_conflict = df[df['SoD'] + df['BPV'] != df['Total Sell-in']]
df_conflict

Промо в df_conflict - всегда

ВОПРОC: Почему если промо был с нулевым содом, то BPV != Total Sell In и что в таких случаях делать?

In [ ]:
df_conflict[df_conflict['SoD'] == 0]

ВОПРОС: Почему здесь не выполняется условие и как его обрабатывать ?

In [ ]:
df_conflict[df_conflict['SoD'] != 0]

In [ ]:
df_conflict[df_conflict['SoD'] > df_conflict['Total Sell-in']]

In [ ]:
exceeding_rows = df_conflict[df_conflict['SoD'] > df_conflict['Total Sell-in']]
average_exceedance = (exceeding_rows["SoD"] / exceeding_rows["Total Sell-in"]).mean()

print(f"SoD превышает Total Sell In: {exceeding_rows.shape[0] / df_conflict.shape[0]:.2%}")
print(f"SoD превышает Total Sell In в среднем: {average_exceedance:.2f} раз")

# Построим еще общий график и посмотрим как себя вели total sell in и bpv на протяжении всего времени

In [ ]:
df.info()

## Как в целом менялся total sell in и bpv с течением времени

In [ ]:
import plotly.express as px

plot_data = df.sort_values(by="Period")

fig1 = px.line(plot_data[plot_data['Promo'] == 0], x="Period", y=["BPV", "Total Sell-in"], 
              labels={"value": "Значение", "Period": "Дата"},
              title="Изменение BPV и Total Sell-in во времени, не было промо")

fig2 = px.line(plot_data[plot_data['Promo'] != 0], x="Period", y=["BPV", "Total Sell-in"], 
              labels={"value": "Значение", "Period": "Дата"},
              title="Изменение BPV и Total Sell-in во времени, было промо")

fig1.show()
fig2.show()

In [ ]:
plot_data[plot_data['Promo'] != 0]['Customer'].value_counts()

In [ ]:
import plotly.express as px

fig1 = px.line(plot_data[(plot_data['Promo'] == 0) & (plot_data['Customer'] == 1)], x="Period", y=["BPV", "Total Sell-in"], 
              labels={"value": "Значение", "Period": "Дата"},
              title="Customer 1: Изменение BPV и Total Sell-in во времени, не было промо")

fig2 = px.line(plot_data[(plot_data['Promo'] != 0) & (plot_data['Customer'] == 1)], x="Period", y=["BPV", "Total Sell-in"], 
              labels={"value": "Значение", "Period": "Дата"},
              title="Customer 1: Изменение BPV и Total Sell-in во времени, было промо")

fig1.show()
fig2.show()

In [ ]:
sales_df_cleaned.sort_values(by="Period")

In [ ]:
import plotly.graph_objects as go

customer_data = sales_df_cleaned[sales_df_cleaned['Customer'] == 1]

dfu_list = customer_data['DFU'].unique()

for dfu in dfu_list:
    dfu_data = customer_data[customer_data['DFU'] == dfu]
    
    dfu_data = dfu_data.sort_values(by='Period')
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dfu_data['Period'], y=dfu_data['BPV'], mode='lines+markers', name='BPV',
        line=dict(color='blue', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=dfu_data['Period'], y=dfu_data['Total Sell-in'], mode='lines+markers', name='Total-sell-in',
        line=dict(color='green', width=2)
    ))
    
    zero_bpv_points = dfu_data[dfu_data['BPV'] == 0]
    fig.add_trace(go.Scatter(
        x=zero_bpv_points['Period'], y=zero_bpv_points['BPV'], mode='markers',
        name='BPV = 0', marker=dict(color='red', size=8)
    ))
    
    min_date = dfu_data['Period'].min()
    
    fig.update_layout(
        title=f'Customer 1, DFU {dfu}<br>BPV, Total-sell-in, and SoD with BPV = 0 highlighted',
        xaxis_title='Period',
        yaxis_title='Values',
        template='plotly_white',
        width=1500,
        height=600,
        legend=dict(x=0.02, y=0.98),
        hovermode='x unified'
    )
    
    fig.update_xaxes(range=['2017-01-01', '2021-07-15'])
    
    fig.show()

In [ ]:
import plotly.graph_objects as go

customer_list = sales_df_cleaned['Customer'].unique()

for customer in customer_list:
    customer_data = sales_df_cleaned[sales_df_cleaned['Customer'] == customer]
    
    dfu_list = customer_data['DFU'].unique()

    for dfu in dfu_list:
        dfu_data = customer_data[customer_data['DFU'] == dfu]
        
        dfu_data = dfu_data.sort_values(by='Period')
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=dfu_data['Period'], y=dfu_data['BPV'], mode='lines+markers', name='BPV',
            line=dict(color='blue', width=2)
        ))
        fig.add_trace(go.Scatter(
            x=dfu_data['Period'], y=dfu_data['Total Sell-in'], mode='lines+markers', name='Total-sell-in',
            line=dict(color='green', width=2)
        ))
        
        zero_bpv_points = dfu_data[dfu_data['BPV'] == 0]
        fig.add_trace(go.Scatter(
            x=zero_bpv_points['Period'], y=zero_bpv_points['BPV'], mode='markers',
            name='BPV = 0', marker=dict(color='red', size=8)
        ))
        
        fig.update_layout(
            title=f'Customer {customer}, DFU {dfu}<br>BPV, Total-sell-in, and SoD with BPV = 0 highlighted',
            xaxis_title='Period',
            yaxis_title='Values',
            template='plotly_white',
            width=1500,
            height=600,
            legend=dict(x=0.02, y=0.98),
            hovermode='x unified'
        )
        
        fig.update_xaxes(range=['2017-01-01', '2021-07-15'])
        
        fig.show()

In [ ]:
sales_df_cleaned = sales_df_cleaned[sales_df_cleaned['Period'] >= '2018-04-23']
sales_df_cleaned

In [ ]:
customer_products = sales_df_cleaned.groupby('Customer')['DFU'].unique().reset_index()
customer_products

In [ ]:
last_sales_dates = sales_df_cleaned.groupby(['DFU', 'Customer'])['Period'].max().reset_index()

min_last_period = last_sales_dates['Period'].min()

earliest_products = last_sales_dates[last_sales_dates['Period'] == min_last_period]

next_10_earliest = (
    last_sales_dates[last_sales_dates['Period'] > min_last_period]
    .sort_values('Period')
    .head(10)
)

final_df = pd.concat([earliest_products, next_10_earliest])

final_df

In [ ]:
dfu_customer_counts = sales_df_cleaned.groupby(['DFU', 'Customer']).size().reset_index(name='Record Count')

final_df = final_df.merge(dfu_customer_counts, on=['DFU', 'Customer'], how='left')


С правой стороны кикаем:

Рис круглозерный 1000 гр, 1
Булгур 300 гр, 1
Рис басмати 500 гр,18
Йогурт Постный 180 гр, 1 ? потенциально - дропаем 72 записи чтобы получить 90, но при этом отказываемся от dfu

максимальноая дата по которой берем датасет - 07.06.2021




что делать с редкими продажами??